In [1]:
import torch.nn as nn
import torch.nn.functional as F
import os
from torchvision import transforms,datasets
from torch.utils.data import Dataset, DataLoader
# import torch.utils.data
import torch.optim as optim
import torch
import cv2
from PIL import Image
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

![](picture/1.png)

In [3]:
class LRN(nn.Module):
    def __init__(self, local_size=1, alpha=1.0, beta=0.75, ACROSS_CHANNELS=True):
        super(LRN, self).__init__()
        self.ACROSS_CHANNELS = ACROSS_CHANNELS
        if ACROSS_CHANNELS:
            self.average=nn.AvgPool3d(kernel_size=(local_size, 1, 1),
                    stride=1,
                    padding=(int((local_size-1.0)/2), 0, 0))
        else:
            self.average=nn.AvgPool2d(kernel_size=local_size,
                    stride=1,
                    padding=int((local_size-1.0)/2))
        self.alpha = alpha
        self.beta = beta


    def forward(self, x):
        if self.ACROSS_CHANNELS:
            div = x.pow(2).unsqueeze(1)
            div = self.average(div).squeeze(1)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        else:
            div = x.pow(2)
            div = self.average(div)
            div = div.mul(self.alpha).add(1.0).pow(self.beta)
        x = x.div(div)
        return x


In [4]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet,self).__init__()

        #conv
        self.conv1 = nn.Conv2d(3,96,kernel_size=11,stride=4,padding = 0)
        self.conv2 = nn.Conv2d(96,256,kernel_size=5,stride=1,padding=2)
        self.conv3 = nn.Conv2d(256,384,kernel_size=3,stride=1,padding=1)
        self.conv4 = nn.Conv2d(384,384,kernel_size=3,stride=1,padding=1)
        self.conv5 = nn.Conv2d(384,256,kernel_size=3,stride=1,padding=1)

        #LRN
        self.LRN = LRN(local_size=5,alpha=0.0001,beta=0.75)

        #FC
        self.fc1 = nn.Linear(256*6*6,4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,2)

        #Dropout
        self.Dropout = nn.Dropout()#Dropout的是为了防止过拟合而设置
        #nn.Dropout(p = 0.3) # 表示每个神经元有0.3的可能性不被激活

    def forward(self,x):
        # conv1 -> relu -> maxpool1
        # conv1: [n, 3, 227, 227] --> [n, 96, 55, 55]
        # maxpool1: [n, 96, 55, 55] --> [n, 96, 27, 27]
        # print(x)
        m = nn.MaxPool2d(3, stride=2)
        
        x = F.relu(self.conv1(x))#第一层卷积结果并激励
        # print('第一个卷积层输出x shape{}'.format(x.shape))
        x = self.LRN(x)
        # print('第一个LRN层输出x shape{}'.format(x.shape))
        # print(x.shape)
        # print(x)
        # x = F.max_pool2d(x,(3,3),2)
        x = m(x)
        # print('第一个池化层输出x shape{}'.format(x.shape))
        # x = F.max_pool2d(x,(3,3),2)

        # conv2 -> relu -> maxpool2
        # conv2: [n, 96, 27, 27] --> [n, 256, 27, 27]
        # maxpool2: [n, 256, 27, 27] --> [n, 256, 13, 13]

        x = F.relu(self.conv2(x))
        # print('第二个卷积层输出x shape{}'.format(x.shape))
        x = self.LRN(x)
        # print('第二个LRN输出x shape{}'.format(x.shape))
        # m = nn.MaxPool2d(3, stride=2)
        x = m(x)
        # print('第二个池化层输出x shape{}'.format(x.shape))
        # x = F.max_pool2d(x,(3,3),2)
            
        # conv3 -> relu -> conv4 -> relu
        # oonv3: [n, 256, 13, 13] --> [n, 384, 13, 13]
        # conv4: [n, 384, 13, 13] --> [n, 384, 13, 13]

        x = F.relu(self.conv3(x))
        # print('第三个卷积层输出x shape{}'.format(x.shape))
        x = F.relu(self.conv4(x))
        # print('第四个卷积层输出x shape{}'.format(x.shape))
        
        # conv5 -> relu -> maxpool3
        # conv5: [n. 384, 13, 13] --> [n, 256, 13, 13]
        # maxpool3: [n, 256, 13, 13] --> [n, 256, 6, 6]
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, (3, 3), 2)
            
        # need view first for conv --> FC
        x = x.view(x.size()[0],-1)#转换维度
        # print('平展后输出x shape{}'.format(x.shape))

        # fc1 -> fc2 -> fc3 -> softmax
        # fc1: 256*6*6 --> 4096
        # fc2: 4096 --> 4096
        # fc3: 1000 --> 2
        # print(x.shape)
        x = F.relu(self.fc1(x))
        x = self.Dropout(x)
        x = F.relu(self.fc2(x))
        x = self.Dropout(x)
        x = self.fc3(x)
        x = F.softmax(x)
        return x


In [5]:
#数据预处理
pre_transforms = transforms.Compose([#一组操作组合起来
    transforms.Resize((227,227)),
    transforms.ToTensor(),#转换类型
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [6]:
class CatDogDataset(Dataset):
    def __init__(self,args,mode = 'train',transform = None):
        self.args = args
        self.transform = transform
        self.mode = mode
        self.names = self.__dataset_info()
    def __getitem__(self,index):#读取一个数据
        x = cv2.imread(self.args+'/'+self.names[index],1)
        #x 为numpy ndarry
        x = Image.fromarray(x)#PIL
        # array -> image
        x_label = 0 if 'cat' in self.names[index] else 1 #贴标签
        if self.transform is not None:
            x = self.transform(x)
        return x,x_label
    
    def __len__(self):
        return len(self.names)
    def __dataset_info(self):#读取数据
        img_path = self.args
        imgs = [f for f in os.listdir(img_path) if
                os.path.isfile(os.path.join(img_path,f)) and f.endswith('.jpg')]
    #listdir 返回当前文件夹下的文件名
    #os.path.join 将目录和文件名合并成一个路径
    #os.path.isfile 判断是否为文件
    #f为字符串
        names = []
        for name in imgs:
            index = int(name.split('.')[1])
            #train dataset
            if self.mode == 'train':
                if index >= 500:
                    names.append(name)
            #test dataset:1000 cat1 dog1
            elif self.mode == 'test':
                if index < 500:
                    names.append(name)
        return names

In [7]:
# 实例化 从训练数据中，划分训练集和验证集
train_dataset = CatDogDataset('dogs-vs-cats/train/','train',pre_transforms)
test_dataset = CatDogDataset('dogs-vs-cats/train/','test',pre_transforms)

#print the length of train_dataset
print('train:{} img'.format(len(train_dataset)))
print('test:{} img'.format(len(test_dataset)))

train:24000 img


In [9]:
batch_size1 = 100
batch_size2 = 10
train_loader = DataLoader(train_dataset,batch_size,shuffle = True)
test_loader = DataLoader(test_dataset,batch_size2,shuffle = False)

In [10]:
alexnet = AlexNet()
#损失函数与优化方法
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(alexnet.parameters(),#momentum 动量
                lr = 0.01,momentum = 0.9,weight_decay = 5e-4)

In [12]:
if torch.cuda.is_available():
    print('Yes')

In [13]:
#选择加载到哪个设备上
#选择运行在哪个设备上
# if :
#     device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#     print('using CPU!')
# else:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('using GPU')
else:
    device = torch.device('cpu')
    print('using CPU')


using CPU


In [14]:
def validate(valid_loader, model, criterion,device):
    '''
    Function for the validation step of the training loop
    '''

    model.eval()
    running_loss = 0

    for X, y_true in valid_loader:
        X = X.to(device)
        y_true = y_true.to(device)
        # Forward pass and record loss
        y_hat = model(X)
        y_hat = y_hat.to(device)
        loss = criterion(y_hat, y_true)
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(valid_loader.dataset)
    return model, epoch_loss


In [15]:
def train(train_loader, model, criterion, optimizer,device):
    '''
    Function for the training step of the training loop
    '''
    model.train()
    runnin_loss = 0

    for X, y_true in train_loader:
        X = X.to(device)
        y_true = y_true.to(device)
        optimizer.zero_grad()

        # Forward: getting the predictions for the batch using current weight
        y_hat = model(X)
        y_hat = y_hat.to(device)
        # Calculate the value of the loss function
        loss = criterion(y_hat, y_true)
        runnin_loss += loss.item() * X.size(0)

        # Backward: The weights are adjusted based on the loss
        loss.backward()
        optimizer.step()
        
    epoch_loss = runnin_loss / len(train_loader.dataset)
    #每一轮训练的平均loss
    return model, optimizer, epoch_loss

In [16]:
def plot_losses(train_losses,valid_losses):
    plt.style.use('seaborn')
    train_losses = np.array(train_losses)
    valid_losses = np.array(valid_losses)
    
    fig,ax = plt.subplots(figsize = (8,5))
    ax.plot(train_losses,color = 'blue',label = 'Training loss')#用默认x轴
    ax.plot(valid_losses,color = 'red',label = 'Validation loss')
    ax.set(title = 'Loss per epochs',xlabel = 'Epoch',ylabel = 'Loss')
    ax.legend()
    fig.show()
    
    plt.style.use('default')
    return None

In [23]:
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs,device,print_every=3):
    '''
    Function defining the entire training loop
    '''
    # set objects for storing metrics
    best_loss = 1e10
    train_losses = []
    valid_losses = []
 
    # Train model
    for epoch in range(0, epochs):

        # training
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer,device)
        train_losses.append(train_loss)

        # validation
        with torch.no_grad():
            model, valid_loss = validate(valid_loader, model, criterion,device)
            valid_losses.append(valid_loss)

        if epoch % print_every == (print_every - 1):            
#             train_acc = get_accuracy(model, train_loader, device=device)
            valid_acc = get_accuracy(model, valid_loader, device=device)
                
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Valid loss: {valid_loss:.4f}\t'
#                   f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Valid accuracy: {100 * valid_acc:.2f}')
            print('#')

    plot_losses(train_losses, valid_losses)
    return model, optimizer, (train_losses, valid_losses)

In [18]:
EPOCHS = 10

In [ ]:
#开始训练
model, optimizer, _ = training_loop(alexnet, criterion, optimizer, train_loader, test_loader, EPOCHS,device)

tensor([0.4066, 0.5934])